<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/07_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 PyTorch入門 (3)
* IMDbデータセットの感情分析をPyTorchを使っておこなう。
 * 前にscikit-learnを使って同じ作業をおこなった。
* 参考資料
 * PyTorch公式のチュートリアル https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
* データは以前作ったIMDbの文書埋め込みを使う。
* sentiment analysisのもっと高度な手法については、下記リンク先を参照。
 * https://github.com/bentrevett/pytorch-sentiment-analysis

## 1. fastTextによる文書埋め込みをMLPの入力として使うための準備
* MLP(多層パーセプトロン)の学習ぐらいは、空気を吸ったり吐いたりするぐらい自然にできるようにしておこう。

### データファイルが置いてあるGoogle Driveのパスを変数PATHに設定
* データファイルの扱い方
 * Blackboardで「自然言語処理特論」へ行く。
 * 「教材/課題/テスト」→「data」→「IMDb」と順にクリックする。
 * 見えている4つの「.npy」ファイルをダウンロードする。
 * ダウンロードした4つのファイルを、自分のGoogle Driveの適当な場所にアップロードする。
 * 次のセルで、その置き場所を指定する。

In [1]:
PATH = '/content/drive/MyDrive/2021Courses/NLP/'

あらかじめランタイムをGPUに設定しておこう

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

np.random.seed(123)
torch.manual_seed(123)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

### 単語埋め込みデータファイルの読み込み

In [4]:
texts = dict()
labels = dict()
for tag in ['train', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [5]:
for tag in ['train', 'test']:
  print(texts[tag].shape)

(25000, 300)
(25000, 300)


### ndarrayをPyTorchのテンソルに変換

In [6]:
for tag in ['train', 'test']:
  texts[tag], labels[tag] = torch.tensor(texts[tag]), torch.tensor(labels[tag])

## 2. 学習のための準備

### 訓練データの分割
* validation setを切り出すため。

In [8]:
from torch.utils.data import Dataset, random_split

class MyDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, index):
    return self.X[index], self.y[index]

train_valid = MyDataset(texts['train'], labels['train'])
test = MyDataset(texts['test'], labels['test'])

valid_size = len(train_valid) // 5
train_size = len(train_valid) - valid_size
train, valid = random_split(train_valid,
                            [train_size, valid_size],
                            generator=torch.Generator().manual_seed(42)
                            )

### データローダの作成
* shuffleをTrueにして、毎エポック異なる順で訓練データを見ていくようにする。

In [9]:
from torch.utils.data import DataLoader

# ミニバッチのサイズ
BATCH_SIZE = 100

# 訓練データだけシャッフル
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid, batch_size=BATCH_SIZE)
test_loader = DataLoader(test, batch_size=BATCH_SIZE)

## 3. モデルの定義と学習の準備

### モデルの定義

In [10]:
class TextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class):
    super(TextSentiment, self).__init__()
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [11]:
EMBED_DIM = texts['train'].shape[1]
NUM_CLASS = len(np.unique(labels['train']))
model = TextSentiment(EMBED_DIM, NUM_CLASS).to(device)

### 損失関数とoptimizerとschedulerを作る
* https://pytorch.org/docs/stable/optim.html

* 損失関数を除いて、以下の設定は適当なので、各自、調整すること。

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,50], gamma=0.1)

## 4. 分類器の訓練と評価

### 評価用の関数
* 正解率で評価する関数を定義しておく。

In [13]:
def eval(loader):
  total_loss = 0.0
  total_acc = 0.0
  total_size = 0
  for input, target in loader:
    with torch.no_grad():
      input, target = input.to(device), target.to(device)
      output = model(input)
      loss = criterion(output, target)
      total_loss += loss.item() * len(target)
      total_acc += (output.argmax(1) == target).float().sum().item()
      total_size += len(target)

  return total_loss / total_size, total_acc / total_size

### 訓練のループ
* 前回とほぼ同じ。

In [14]:
def train(train_loader, valid_loader, n_epochs=100):
  # training loop
  for epoch in range(n_epochs):
    train_loss = 0.0
    for input, target in train_loader:
      optimizer.zero_grad()
      # forward pass
      output = model(input.to(device))
      loss = criterion(output, target.to(device))
      train_loss += loss.item() * len(target) # 表示用の集計
      # backward pass
      loss.backward()
      # update parameters
      optimizer.step()

    # valid loss
    valid_loss = 0
    with torch.no_grad():
      for input, target in valid_loader:
        output = model(input.to(device))
        loss = criterion(output, target.to(device))
        valid_loss += loss.item() * len(target)

    # logging
    print(f'epoch {epoch + 1:4d} ;',
          f'train loss {train_loss / train_size:8.4f} ;',
          f'valid loss {valid_loss / valid_size:8.4f} ;')

In [15]:
train(train_loader, valid_loader, 100)

epoch    1 ; train loss   0.6448 ; valid loss   0.6136 ;
epoch    2 ; train loss   0.5947 ; valid loss   0.5793 ;
epoch    3 ; train loss   0.5659 ; valid loss   0.5566 ;
epoch    4 ; train loss   0.5450 ; valid loss   0.5400 ;
epoch    5 ; train loss   0.5305 ; valid loss   0.5303 ;
epoch    6 ; train loss   0.5174 ; valid loss   0.5156 ;
epoch    7 ; train loss   0.5052 ; valid loss   0.5095 ;
epoch    8 ; train loss   0.4946 ; valid loss   0.5028 ;
epoch    9 ; train loss   0.4843 ; valid loss   0.4835 ;
epoch   10 ; train loss   0.4755 ; valid loss   0.4769 ;
epoch   11 ; train loss   0.4691 ; valid loss   0.4670 ;
epoch   12 ; train loss   0.4614 ; valid loss   0.4604 ;
epoch   13 ; train loss   0.4568 ; valid loss   0.4933 ;
epoch   14 ; train loss   0.4491 ; valid loss   0.4811 ;
epoch   15 ; train loss   0.4462 ; valid loss   0.4471 ;
epoch   16 ; train loss   0.4404 ; valid loss   0.4427 ;
epoch   17 ; train loss   0.4381 ; valid loss   0.4366 ;
epoch   18 ; train loss   0.431

* テストセット上での正解率を調べてみる。

In [18]:
test_loss, test_acc = eval(test_loader)
print(f'test loss : {test_loss:8.4f} | test accuracy : {test_acc:.3f}')

test loss :   0.3626 | test accuracy : 0.844


# 課題7
* モデルやoptimizerやschedulerを変更して、dev setの分類性能をできるだけ向上させてみよう。
* その後、自分で選択した設定を使って、最終的にtest setで評価しよう。